We will use data from *Bees of Guam* as a test case.

http://tb.plazi.org/GgServer/dwca/FE566D11FFD2FFF5383F9056FFE3FFEF.zip

In [7]:
import pandas as pd
from dwca.read import DwCAReader
import os

In [8]:
uuid = 'FE566D11FFD2FFF5383F9056FFE3FFEF'

In [9]:
dwca_url = f'http://tb.plazi.org/GgServer/dwca/{uuid}.zip'
dwca_file = f'{uuid}.zip'

# download the DwCA into the current working directory, overwriting previous DwCA with same uuid
os.system(f'wget -O {dwca_file} {dwca_url}')

# unzip the DwCA, overwriting files
os.system(f'unzip -o {dwca_file}')
  
    
    
    
#     with DwCAReader(dwca_file) as dwca:
#         print(dwca.absolute_temporary_path('occurrence.txt'))
#         df_myoccurrences = pd.read_csv(dwca.absolute_temporary_path('occurrence.txt'))
#         df_occurrences = dwca.pd_read('occurrences.txt', parse_dates=True)
#         df_taxa = dwca.pd_read('taxa.txt')
#         df_description = dwca.pd_read('description.txt')
#     return {'taxa':df_taxa, 'occurrences':df_occurrences, 'description':df_description}

# data = get_dwca('FE566D11FFD2FFF5383F9056FFE3FFEF')

0

In [20]:
taxon_df = pd.read_csv('taxa.txt', sep='\t')
taxon_df.head(1)

,taxonID,namePublishedIn,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,kingdom,phylum,class,order,family,...,scientificNameAuthorship,canonicalName,verbatimScientificName,basionymAuthors,basionymYear,combinationAuthors,combinationYear,taxonomicStatus,nomenclaturalStatus,references
0,026F1569FFD3FFF439229473FCE8F968.taxon,NaN,NaN,NaN,NaN,Animalia,Arthropoda,Insecta,Hymenoptera,Megachilidae,...,(Cockerell),Hylaeus guamensis,Hylaeus guamensis (Cockerell),Cockerell,NaN,NaN,NaN,NaN,NaN,http://treatment.plazi.org/id/026F1569FFD3FFF4...


In [19]:
occ_df = pd.read_csv('occurrences.txt', sep='\t')
occ_df.head(1)

,occurrenceID,taxonID,catalogNumber,collectionCode,institutionCode,typeStatus,verbatimLabel,sex,individualCount,eventDate,...,minimumElevationInMeters,maximumElevationInMeters,minimumDepthInMeters,maximumDepthInMeters,country,stateProvince,municipality,locality,references,basisOfRecord
0,026F1569FFD3FFF439229473FCE8F968.mc.3AAEAE22FF...,026F1569FFD3FFF439229473FCE8F968.taxon,NaN,NaN,NaN,NaN,"Sumay Road, June 23, Swezey.",NaN,1,1936-06-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sumay Road,http://treatment.plazi.org/id/026F1569FFD3FFF4...,PreservedSpecimen


In [34]:
for i, taxon in taxon_df.iterrows():
    print(taxon.canonicalName)
    for i, occ in occ_df[occ_df.taxonID==taxon.taxonID].iterrows():
        print(f'   {occ.country}   {occ.locality}   {occ.recordedBy}   {occ.eventDate}')

Hylaeus guamensis
   nan   Sumay Road   O. H. Swezey   1936-06-23
Lithurgus guamensis
   nan   Tarague Beach   O. H. Swezey   1936-05-17
   nan   Machanao   R. L. Usinger & O. H.Swezey   1936-06-30
   nan   Orote Peninsula   O. H. Swezey   1936-09-27
   nan   Yigo   O. H. Swezey   1936-05-19
   nan   Yigo   O. H. Swezey   1936-10-18
Megachile fullawayi
Megachile laticeps
   nan   Tarague Beach   E. H. Bryan   1936-04-16
   nan   Agana   Rowley   1936-08-13
   nan   Barrigada   O. H. Swezey   1936-06
   nan   Piti   O. H. Swezey   1936-05-12
   nan   Piti   O. H. Swezey   1936-07-07
   nan   Piti   O. H. Swezey   1936-07-04
   nan   Piti   O. H. Swezey   1936-09-16
Halictus swezeyi
   nan   Ritidian Point   E. H. Bryan   1936-04-16
   nan   Mt. Sasalaguan   E. H. Bryan   1936-04-25
   nan   Yona   E. H. Bryan   1936-04-29
   nan   Dededo   O. H. Swezey   1936-05-11
   nan   Santa Rosa Peak   O. H. Swezey   1936-05-19
   nan   Merizo   O. H. Swezey & R. L. Usinger   1936-06-11
   nan   A

In [32]:
taxon_df.columns

Index(['taxonID', 'namePublishedIn', 'acceptedNameUsageID',
       'parentNameUsageID', 'originalNameUsageID', 'kingdom', 'phylum',
       'class', 'order', 'family', 'genus', 'taxonRank', 'scientificName',
       'scientificNameAuthorship', 'canonicalName', 'verbatimScientificName',
       'basionymAuthors', 'basionymYear', 'combinationAuthors',
       'combinationYear', 'taxonomicStatus', 'nomenclaturalStatus',
       'references'],
      dtype='object')

In [35]:
occ_df.columns

Index(['occurrenceID', 'taxonID', 'catalogNumber', 'collectionCode',
       'institutionCode', 'typeStatus', 'verbatimLabel', 'sex',
       'individualCount', 'eventDate', 'recordedBy', 'recordNumber',
       'decimalLatitude', 'decimalLongitude', 'minimumElevationInMeters',
       'maximumElevationInMeters', 'minimumDepthInMeters',
       'maximumDepthInMeters', 'country', 'stateProvince', 'municipality',
       'locality', 'references', 'basisOfRecord'],
      dtype='object')

In [41]:
with DwCAReader(dwca_file) as dwca:
    print(dwca.absolute_temporary_path('occurrence.txt'))
    df_myoccurrences = pd.read_csv(dwca.absolute_temporary_path('occurrence.txt'))
df_myoccurrences.head(1)

/tmp/tmpghna4d0j/occurrence.txt


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpghna4d0j/occurrence.txt'

In [8]:
data['description'].description.values[3]

'Ritidian Pt., April 16, Bryan; Mt. Sasalaguan, April 25, Bryan; Yona, April 29, Bryan; Dededo, on flowers of unknown shrub, May 11, Swezey; Santa Rosa Peak, May 19, Swezey; Merizo, June 11, Swezey, Usinger; near Atao Beach, June 25 ,, Usinger; Machanao, on tobacco flowers, June 30, Swezey; Dandan, on Glochidion flowers, July 17, Swezey; Piti, Aug. 24, Sept. 27, Swezey.'

In [27]:
dwca_url = f'http://tb.plazi.org/GgServer/dwca/{uuid}.zip'
dwca_file = f'{uuid}.zip'
if os.path.exists(dwca_file):
    os.remove(dwca_file)
    os.system(f'wget {dwca_url}')    

with DwCAReader(dwca_file) as dwca:
    # Check the core file of the Archive  (Occurrence, Taxon, ...)
    print("Core type is: {}".format(dwca.descriptor.core.type))

    # Check the available extensions
    print("Available extensions: {}".format([ext.split("/")[-1] for ext in dwca.descriptor.extensions_type]))

    taxon_df = dwca.pd_read('taxa.txt')
#     taxon_df['id'] = pd.to_numeric(taxon_df['id'])

    print(taxon_df.shape)
    descr_df = dwca.pd_read('description.txt')
    print(descr_df.shape)
    occ_df = dwca.pd_read('occurrences.txt')
    print(occ_df.shape)

# Join the information of the description and vernacularname extension to the core taxon information
# (cfr. database JOIN)
df = pd.merge(taxon_df, occ_df, left_on='id', right_on='coreid', how="left")
print(df.shape)

Core type is: http://rs.tdwg.org/dwc/terms/Taxon
Available extensions: ['Occurrence', 'Description', 'Distribution', 'Document', 'Multimedia', 'Reference', 'VernacularName']
(7, 23)
(5, 5)
(31, 23)


ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat

In [22]:
taxon_df.dtypes

id                           object
namePublishedIn             float64
acceptedNameUsageID         float64
parentNameUsageID           float64
originalNameUsageID         float64
kingdom                      object
phylum                       object
class                        object
order                        object
family                       object
genus                        object
taxonRank                    object
scientificName               object
scientificNameAuthorship     object
canonicalName                object
verbatimScientificName       object
basionymAuthors              object
basionymYear                float64
combinationAuthors           object
combinationYear             float64
taxonomicStatus             float64
nomenclaturalStatus         float64
references                   object
dtype: object

In [37]:
occ_df.columns

Index(['coreid', 'catalogNumber', 'collectionCode', 'institutionCode',
       'typeStatus', 'verbatimLabel', 'sex', 'individualCount', 'eventDate',
       'recordedBy', 'recordNumber', 'decimalLatitude', 'decimalLongitude',
       'minimumElevationInMeters', 'maximumElevationInMeters',
       'minimumDepthInMeters', 'maximumDepthInMeters', 'country',
       'stateProvince', 'municipality', 'locality', 'references',
       'basisOfRecord'],
      dtype='object')

In [24]:
descr_df.dtypes

coreid         object
type           object
description    object
language       object
source         object
dtype: object

In [17]:
df.columns

Index(['id', 'namePublishedIn', 'acceptedNameUsageID', 'parentNameUsageID',
       'originalNameUsageID', 'kingdom', 'phylum', 'class', 'order', 'family',
       'genus', 'taxonRank', 'scientificName', 'scientificNameAuthorship',
       'canonicalName', 'verbatimScientificName', 'basionymAuthors',
       'basionymYear', 'combinationAuthors', 'combinationYear',
       'taxonomicStatus', 'nomenclaturalStatus', 'references', 'coreid',
       'type', 'description', 'language', 'source'],
      dtype='object')

In [20]:
df[['scientificName', 'description']]

,scientificName,description
0,Hylaeus guamensis (Cockerell),"Sumay Road, June 23, Swezey."
1,Lithurgus guamensis Cockerell,"Tarague Beach, on cotton flowers, May 17, Swez..."
2,Megachile fullawayi Cockerell,NaN
3,Megachile laticeps Smith,"Ritidian Pt., April 16, Bryan; Agana, Aug. 13,..."
4,Halictus swezeyi Cockerell,"Ritidian Pt., April 16, Bryan; Mt. Sasalaguan,..."
5,Apis mellifera Linnaeus,NaN
6,Halictus saffordi Cockerell,"Orote Peninsula, April 7, on coconut blossoms,..."


In [31]:
taxon_df.head(1)

,id,namePublishedIn,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,kingdom,phylum,class,order,family,...,scientificNameAuthorship,canonicalName,verbatimScientificName,basionymAuthors,basionymYear,combinationAuthors,combinationYear,taxonomicStatus,nomenclaturalStatus,references
0,026F1569FFD3FFF439229473FCE8F968.taxon,NaN,NaN,NaN,NaN,Animalia,Arthropoda,Insecta,Hymenoptera,Megachilidae,...,(Cockerell),Hylaeus guamensis,Hylaeus guamensis (Cockerell),Cockerell,NaN,NaN,NaN,NaN,NaN,http://treatment.plazi.org/id/026F1569FFD3FFF4...


In [36]:
print(occ_df)

                                                                                           coreid  \
026F1569FFD3FFF439229473FCE8F968.mc.3AAEAE22FFD... 026F1569FFD3FFF439229473FCE8F968.taxon     NaN   
026F1569FFD3FFF4392392F2FC1CFC72.mc.3AAEAE22FFD... 026F1569FFD3FFF4392392F2FC1CFC72.taxon     NaN   
026F1569FFD3FFF4392392F2FC1CFC72.mc.3AAEAE22FFD... 026F1569FFD3FFF4392392F2FC1CFC72.taxon     NaN   
026F1569FFD3FFF4392392F2FC1CFC72.mc.3AAEAE22FFD... 026F1569FFD3FFF4392392F2FC1CFC72.taxon     NaN   
026F1569FFD3FFF4392392F2FC1CFC72.mc.3AAEAE22FFD... 026F1569FFD3FFF4392392F2FC1CFC72.taxon     NaN   
026F1569FFD3FFF4392392F2FC1CFC72.mc.3AAEAE22FFD... 026F1569FFD3FFF4392392F2FC1CFC72.taxon     NaN   
026F1569FFD2FFF4398A9638FE49FE76.mc.3AAEAE22FFD... 026F1569FFD2FFF4398A9638FE49FE76.taxon     NaN   
026F1569FFD2FFF4398A9638FE49FE76.mc.3AAEAE22FFD... 026F1569FFD2FFF4398A9638FE49FE76.taxon     NaN   
026F1569FFD2FFF4398A9638FE49FE76.mc.3AAEAE22FFD... 026F1569FFD2FFF4398A9638FE49FE76.taxon  